In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from sklearn.cluster import SpectralClustering
import psutil
import os
import random
import torch
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [2]:
  

def generate_parametric_product_graph(s00, s01, s10, s11, A_T, A_N, spatial_graph):
    #print("Generating product graph...")
    I_T = np.identity(len(A_T))
    I_N = np.identity(len(A_N))
    S_diamond = (
        s00 * np.kron(I_T, I_N) +
        s01 * np.kron(I_T, A_N) +
        s10 * np.kron(A_T, I_N) +
        s11 * np.kron(A_T, A_N)
    )
    product_graph = nx.from_numpy_array(S_diamond)
    
    # Add features to the product graph
    num_nodes = A_N.shape[0]
    num_timesteps = A_T.shape[0]
    
    for t in range(num_timesteps):
        for node in range(num_nodes):
            original_node = node #node_list[node]  # Adjust if necessary
            new_node = t * num_nodes + node
            product_graph.nodes[new_node]['feature'] = spatial_graph.nodes[original_node]['feature']
    
    #print("Product graph generated")
    return product_graph


def perform_spectral_clustering(n_clusters, adj):
    #print("Clustering...")
    spectral = SpectralClustering(n_clusters=n_clusters, affinity='precomputed', assign_labels='kmeans')
    labels = spectral.fit_predict(adj)
    
    # Generate adjacency matrices for subgraphs based on clustering labels
    subgraph_adj_matrices = []
    original_node_ids = []
    
    for cluster in range(n_clusters):
        nodes_in_cluster = [i for i in range(len(labels)) if labels[i] == cluster]
        subgraph_adj_matrix = adj[np.ix_(nodes_in_cluster, nodes_in_cluster)]
        subgraph_adj_matrices.append(subgraph_adj_matrix)
        if cluster < len(listings):
            node_ids = listings.iloc[nodes_in_cluster]['id'].tolist()
            original_node_ids.append(node_ids)
    
    #print("Clustering completed")
    return subgraph_adj_matrices, original_node_ids



def sample_nodes(adj_matrix, M, k):
    """
    Perform spatial clustering by sampling M nodes and including their top-k neighbors based on edge weights.
    
    Parameters:
    adj_matrix (torch.Tensor): The adjacency matrix of the entire graph (shape: [num_nodes, num_nodes]).
    M (int): The number of nodes to sample.
    k (int): The number of top neighbors to sample based on edge weights.
    
    Returns:
    list: List of node indices of the sampled nodes and their neighbors.
    torch.Tensor: The adjacency matrix of the subgraph.
    """
    num_nodes = adj_matrix.shape[0]
    
    # Sample M unique nodes
    sampled_nodes = np.random.choice(num_nodes, M, replace=False)
    
    # Create a mask for sampled nodes and their top-k neighbors
    node_mask = torch.zeros(num_nodes, dtype=torch.bool)
    node_mask[sampled_nodes] = True
    if isinstance(adj_matrix, np.ndarray):
        adj_matrix = torch.tensor(adj_matrix, dtype=torch.float)
    for node in sampled_nodes:
        # Get the neighbors and their edge weights
        neighbors = torch.nonzero(adj_matrix[node] > 0).view(-1)
        neighbor_weights = adj_matrix[node, neighbors]
        
        # Sort neighbors by edge weight and take the top k
        if neighbors.size(0) > k:
            top_k_neighbors = neighbors[torch.topk(neighbor_weights, k).indices]
        else:
            top_k_neighbors = neighbors
        
        node_mask[top_k_neighbors] = True
    
    # Get the final list of nodes (sampled nodes + their top-k neighbors)
    final_nodes = torch.nonzero(node_mask).view(-1)
    
    # Create the subgraph adjacency matrix
    subgraph_adj_matrix = adj_matrix[final_nodes][:, final_nodes]
    
    return final_nodes.tolist(), subgraph_adj_matrix

def adjacency_to_edge_list(adj):
    edge_list = []
    num_nodes = adj.shape[0]
    for i in range(num_nodes):
        for j in range(num_nodes):
            if adj[i][j] != 0:
                edge_list.append((i,j))
    edge_list = np.array(edge_list)
    return edge_list
    
def create_sequences(listing_ids,obs_window,pred_horizon):
    # nodes is the index of the nodes in the graph
    calendar = pd.read_csv(folder+'preprocessed/calendar.csv')
    calendar['date'] = pd.to_datetime(calendar['date'])
    filtered_calendar = calendar[calendar['listing_id'].isin(listing_ids)]
    price_matrix = filtered_calendar.pivot(index='listing_id', columns='date', values='price')
    X, y = [], []
    for listing_id in price_matrix.index:
        data = price_matrix.loc[listing_id].values
        for i in range(len(data) - T - h + 1):
            X.append(data[i:i+T])
            y.append(data[i+T+h-1])
    return np.array(X), np.array(y)

def add_features_to_graph(node_list, graph, features):
    for i, node in enumerate(node_list):
        graph.nodes[i]['feature'] = {feature: features[feature][node] for feature in features}

In [6]:
#folder = "/home/dlayh/Coding/Graph_Project/ML4G-Project-main/data/"
folder = ""
file = "adjacency/coords_features"
#file = "sigma25"
#file = "sigma2"
adj = np.load(folder+file+'.npy')
calendar = pd.read_csv(folder+'preprocessed/calendar.csv')
listings = pd.read_csv(folder+'preprocessed/listings.csv')
#prices = calendar[['price']]
prices = np.log10(calendar['price'].to_numpy())

PRICE_MEAN = prices.mean()

# Initialize the StandardScaler
# scaler = StandardScaler()

# Fit and transform the "price" column
scaled_prices = prices - PRICE_MEAN

# Convert the result back to a DataFrame and update the original DataFrame
calendar['price'] = scaled_prices
# Initialize the StandardScaler
#scaler = StandardScaler()

# Fit and transform the "price" column
#scaled_prices = scaler.fit_transform(prices)

# Convert the result back to a DataFrame and update the original DataFrame
#calendar['price'] = scaled_prices

pred_horizon = 1
obs_window = 3

# Assuming 4 timesteps and predicting a fifth timestep
A_T = np.array([
    [0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0]
])
# Assuming 3 timesteps and predicting a fourth timestep
A_T = np.array([
    [0, 0, 0, 0],
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0]
])
binary_adj = [1 for y in adj for x in y if x!=0]
print(sum(binary_adj)/adj.shape[0]**2)

#subgraphs_adj,original_node_indices = perform_spectral_clustering(10,adj)
node_list, sampled_adj = sample_nodes(adj, 100, 4)
spatial_graph = nx.from_numpy_array(sampled_adj.numpy())

features = ['minimum_nights','square_feet','bathrooms', 'bedrooms', 'beds', 'bed_type']
node_features = listings[features]

add_features_to_graph(node_list, spatial_graph, node_features)

product_graph = generate_parametric_product_graph(0, 1, 1, 1, A_T, sampled_adj, spatial_graph)

def add_labels(G,node_list,time_steps,starting_date):
    num_nodes = len(G.nodes) 
    #print(num_nodes,len(node_list))
    for t in range(time_steps):
        date_obj = datetime.strptime(starting_date, '%Y-%m-%d')
        new_date_obj = date_obj + timedelta(days=t)
        new_date_str = new_date_obj.strftime('%Y-%m-%d')
        labels = calendar[calendar["date"] == new_date_str]
        for node in range(len(node_list)):
            label = labels.iloc[node_list[node]].price
            original_node = node #node_list[node] 
            new_node = (t * len(node_list)) + node 
            #print(new_node, len(node_list))
            #if new_node >= num_nodes:
            #    break
            G.nodes[new_node]['label'] = label 
    return G

labeled_product_graph = add_labels(product_graph,node_list,obs_window+pred_horizon,starting_date = '2016-12-01')
calendar.head()

0.24207689581117992


,Unnamed: 0,listing_id,date,available,price
0,0,5506,2016-12-01,t,-0.032965
1,1,5506,2016-12-02,t,-0.032965
2,2,5506,2016-12-03,t,-0.032965
3,3,5506,2016-12-04,t,-0.032965
4,4,5506,2016-12-05,t,-0.032965


In [7]:
def tikhonov(lambda_reg, signals, laplacian, matrix):
    matrix = np.diag(matrix)
    A = np.matmul(matrix.T,matrix) + lambda_reg*laplacian
    B = np.matmul(np.matmul(matrix.T,matrix),signals)
    return np.matmul(np.linalg.inv(A),B)

def modified_tikhonov(lambda_reg_space,lambda_reg_time, signals, time_laplacian,space_laplacian, matrix):
    matrix = np.diag(matrix)
    A = np.matmul(matrix.T,matrix) + lambda_reg_space*space_laplacian + lambda_reg_time*time_laplacian
    B = np.matmul(np.matmul(matrix.T,matrix),signals)
    return np.matmul(np.linalg.inv(A),B)


def TV_reg(lambda_reg, signals, laplacian, matrix):
    matrix = np.diag(matrix)
    A = np.matmul(matrix.T,matrix) + lambda_reg*np.matmul((np.identity(len(laplacian))-laplacian),(np.identity(len(laplacian))-laplacian).T)
    B = np.matmul(np.matmul(matrix.T,matrix),signals)
    return np.matmul(np.linalg.inv(A),B)

from sklearn.linear_model import Lasso

def lasso_regression(lambda_reg, y, laplacian, mask):
    L = laplacian
    mask_diag = np.diag(mask)
    
    # Apply the mask to the Laplacian and y
    masked_L = np.matmul(mask_diag, L)
    masked_y = np.matmul(mask_diag, y)
    
    # Lasso regression model
    lasso = Lasso(alpha=lambda_reg, fit_intercept=False, positive=True, max_iter=10000)
    lasso.fit(masked_L, masked_y)
    
    return lasso.coef_
    
    return ridge.coef_
def construct_laplacian(adj_matrix):
    degree_matrix = np.diag(np.sum(adj_matrix, axis=1))
    laplacian = degree_matrix - adj_matrix
    return laplacian
def construct_spatial_laplacian(adj_matrix):
    mask_diag = np.identity(len(adj_matrix))
    adj_matrix = mask_diag * adj_matrix
    degree_matrix = np.diag(np.sum(adj_matrix, axis=1))
    laplacian = degree_matrix - adj_matrix
    return laplacian

def construct_temporal_laplacian(T):
    temporal_graph = nx.path_graph(T)
    adj_matrix = nx.adjacency_matrix(temporal_graph).toarray()
    degree_matrix = np.diag(np.sum(adj_matrix, axis=1))
    laplacian = degree_matrix - adj_matrix
    return laplacian
def modify_laplacian_for_temporal_emphasis(laplacian, N, T):
    I_spatial = np.kron(np.eye(T), np.ones((N, N)))  # Identity matrix for spatial connections
    I_temporal = np.kron(np.ones((T, T)), np.eye(N))  # Identity matrix for temporal connections
    
    lap_space =  laplacian * I_spatial
    lap_temp = laplacian * I_temporal
    return lap_space,lap_temp

def elastic_net_regression(alpha, l1_ratio, y, laplacian, mask):
    mask_diag = np.diag(mask)
    
    # Apply the mask to the Laplacian and y
    masked_L = mask_diag @ laplacian
    masked_y = mask_diag @ y
    
    # Elastic Net regression model
    elastic_net = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, fit_intercept=False, max_iter=10000)
    elastic_net.fit(masked_L, masked_y)
    
    return elastic_net.coef_
def find_best_hyperparameters(y, laplacian, mask):
    mask_diag = np.diag(mask)
    masked_L = mask_diag @ laplacian
    masked_y = mask_diag @ y
    
    # Define the parameter grid
    param_grid = {
        'alpha': np.logspace(-4, 1, 2),
        'l1_ratio': np.linspace(0.0001, 1, 2)
    }
    
    # Create the ElasticNet model
    elastic_net = ElasticNet(fit_intercept=False, max_iter=10000)
    
    # Create the GridSearchCV object
    grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
    
    # Perform the grid search
    grid_search.fit(masked_L, masked_y)
    
    # Get the best parameters
    best_params = grid_search.best_params_
    
    return best_params

In [11]:


def regularize(labeled_product_graph,plot=False):
    
    signals = []
    block_size = len(node_list)+1 # 4 timesteps, 
    N = block_size-1
    T = 4   # Number of timesteps
    for node in labeled_product_graph.nodes():
        try:
            signals.append(labeled_product_graph.nodes[node]['label'])
        except KeyError:
            print(f"Node {node} is missing 'label' attribute.")
    
    signals = np.array(signals)
    adj_matrix = nx.adjacency_matrix(labeled_product_graph).toarray()
    binary_adj_matrix = (adj_matrix > 0).astype(int)
    #laplacian = construct_laplacian(adj_matrix)
    
    laplacian = construct_laplacian(binary_adj_matrix)
    laplacian_spatial,laplacian_temporal = modify_laplacian_for_temporal_emphasis(laplacian, N, T)
    
    #laplacian_spatial = construct_spatial_laplacian(labeled_product_graph)
    #laplacian_temporal = construct_temporal_laplacian(labeled_product_graph)
    # Mask a certain percentage of node signals
    
    # Create a mask where the last block is set to 0 (missing)
    mask = np.ones_like(signals)
    mask[-block_size:] = 0
    #print(mask)
    masked_signals = signals * mask
    alpha = [0]#np.linspace(1e-10,100,20)
    l1_ratio = [1e-20]#np.linspace(1e-10,1,10)
    lambda_reg_spatial = [0]#np.linspace(1e-100,100,20)
    lambda_reg = [0]#np.linspace(1e-100,100,20)
    #for j in alpha:#_temporal:
        #for i in l1_ratio:
    # Mix between L1 and L2 (0.5 means equal contribution)
    
    # Estimate the missing signals using Elastic Net regularization
    #stimated_signals = elastic_net_regression(j, i, masked_signals, laplacian, mask)
    estimated_signals = TV_reg(1e-20, masked_signals, laplacian, mask)
    #estimated_signals = lasso_regression(i, masked_signals, laplacian, mask)
    #estimated_signals = modified_tikhonov(i, j, 
    #                                      masked_signals,  laplacian_temporal,laplacian_spatial, mask)
    # Reconstruct the original signal by replacing the masked indices with the estimated values
    reconstructed_signals = np.copy(signals)
    reconstructed_signals[-block_size:] = estimated_signals[-block_size:]
    
    # Calculate MSE between the original and estimated signals for the last block
    original_signals_last_block = signals[-block_size:]
    estimated_signals_last_block = estimated_signals[-block_size:]
    size = len(original_signals_last_block)
    largest_square_size = int(np.floor(np.sqrt(size))) ** 2
    sqrt_size = int(np.sqrt(largest_square_size))
    original_signals_last_block = original_signals_last_block[:largest_square_size]
    estimated_signals_last_block = estimated_signals_last_block[:largest_square_size]
    original_signals_last_block = np.reshape(original_signals_last_block,(sqrt_size,sqrt_size))
    estimated_signals_last_block = np.reshape(estimated_signals_last_block,(sqrt_size,sqrt_size))
    mse = mean_squared_error(original_signals_last_block, estimated_signals_last_block)
    mae = mean_absolute_error(10**(original_signals_last_block+PRICE_MEAN), 10**(estimated_signals_last_block+PRICE_MEAN))
    r2 = r2_score(original_signals_last_block, estimated_signals_last_block)

    #print("Original Signals in Last Block: ", original_signals_last_block,
    #      sum(original_signals_last_block)/len(original_signals_last_block))
    #print("Masked Signals in Last Block: ", masked_signals[-block_size:])
    #print("Estimated Signals in Last Block: ", estimated_signals_last_block,
    #     sum(estimated_signals_last_block)/len(estimated_signals_last_block))
    #print("Reconstructed Signals: ", reconstructed_signals)
    if plot==True:
        with np.errstate(divide='ignore', invalid='ignore'):
            relative_error_matrix = np.abs((original_signals_last_block - estimated_signals_last_block) / original_signals_last_block) * 100
            relative_error_matrix[original_signals_last_block == 0] = np.nan  # Assign NaN where true_matrix is 0
        vmin, vmax = min(np.min(original_signals_last_block), np.min(estimated_signals_last_block)), max(np.max(original_signals_last_block), np.max(estimated_signals_last_block))
        #, axes = plt.subplots(1, 3, figsize=(16, 8))
        plt.figure(figsize=(6, 6))
        plt.imshow(original_signals_last_block, cmap='viridis', vmin=vmin, vmax=vmax)
        plt.title('True Price')
        plt.colorbar()
        plt.savefig("true_matrix.png", format='png')
        plt.show()
        plt.close()
        
        # Save the predicted matrix plot
        plt.figure(figsize=(6, 6))
        plt.imshow(estimated_signals_last_block, cmap='viridis', vmin=vmin, vmax=vmax)
        plt.title('Predicted Price')
        plt.colorbar()
        plt.savefig("predicted_matrix.png", format='png')
        plt.show()
        plt.close()
        
        # Save the relative error matrix plot
        plt.figure(figsize=(6, 6))
        plt.imshow(relative_error_matrix, cmap='viridis', vmin=0, vmax=100)
        plt.title('Relative Error (%)')
        plt.colorbar()
        plt.savefig("relative_error_matrix.png", format='png')
        plt.show()
        plt.close()
    #print("MSE between original and estimated signals in last block: ", mse)
    #print(sum(original_signals_last_block.flatten())/len(original_signals_last_block.flatten()))
    #print(sum(estimated_signals_last_block.flatten())/len(estimated_signals_last_block.flatten()))
    return mse,mae,r2

mse_s=[]
mae_s=[]
r2_s = []
starting_dates = list(set(calendar.date))
#print(starting_dates[:10])
for i in range(1):
    for date in tqdm(starting_dates,total=len(starting_dates)):
        node_list, sampled_adj = sample_nodes(adj, 100, 4)
        spatial_graph = nx.from_numpy_array(sampled_adj.numpy())
        
        features = ['minimum_nights','square_feet','bathrooms', 'bedrooms', 'beds', 'bed_type']
        node_features = listings[features]
        
        add_features_to_graph(node_list, spatial_graph, node_features)
        
        product_graph = generate_parametric_product_graph(0, 1, 1, 1, A_T, sampled_adj, spatial_graph)
        labeled_product_graph = add_labels(product_graph,node_list,obs_window+pred_horizon,starting_date = '2016-12-01')
        mse,mae,r2 = regularize(labeled_product_graph)
        mse_s.append(mse)
        mae_s.append(mae)
        r2_s.append(r2)
print(sum(mse_s)/len(mse_s))
print(sum(mae_s)/len(mae_s))
print(sum(r2_s)/len(r2_s))

100%|██████████████████████| 279/279 [12:10<00:00,  2.62s/it]

0.09088180370123376
615.7449270062571
-0.20252415827279485


In [ ]:
def test_mse(a,b):
    mse =0
    for i in range(len(a)):
        mse += (a[i]-b[i])**2
    return mse/len(a)
print(test_mse(original_signals_last_block,estimated_signals_last_block))

In [ ]:
starting_date = '2016-12-01'
date_obj = datetime.strptime(starting_date, '%Y-%m-%d')
t = 1
new_date_obj = date_obj + timedelta(days=t)
new_date_str = new_date_obj.strftime('%Y-%m-%d')
print(new_date_str)
temp = calendar[calendar["date"] == new_date_str]
temp.head()
temp.loc[280].price

In [8]:
with open('edge_list.txt', 'w') as f:
    for edge in edge_list:
        f.write(f"{edge[0]} {edge[1]}\n")

In [4]:
loaded_edge_list = []
with open('edge_list.txt', 'r') as f:
    for line in f:
        node1, node2 = map(int, line.split())
        loaded_edge_list.append((node1, node2))
G = nx.Graph()
G.add_edges_from(loaded_edge_list)
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
is_directed = G.is_directed()

print(f"Graph with {num_nodes} nodes and {num_edges} edges")
print(f"Directed: {is_directed}")

Graph with 10516 nodes and 7606391 edges
Directed: False


AttributeError: module 'networkx' has no attribute 'info'

In [5]:
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G_loaded)
nx.draw(G_loaded, pos, with_labels=True, node_color='lightblue', node_size=500, font_size=10)
plt.title("Loaded Graph from Edge List")
plt.show()

KeyboardInterrupt: 

<Figure size 800x600 with 0 Axes>